In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

pp=pd.read_csv('E:\\Anaconda3\\PyComDH\\1.Pipe.csv',header=0,encoding='gbk')
ppdia=pd.concat([pp.loc[range(0,68),['内径']]*0.01*i for i in range(5,96)],axis=0).reset_index()  #设置管径
ppdia=ppdia['内径']
ppcoe=pd.concat([pd.DataFrame(np.ones([68,1]))*0.01*i for i in range(5,96)],axis=0).reset_index()#生成管径系数
ppcoe=ppcoe[0]
ppcode=(pd.concat([pp.loc[range(0,68),['编号']] for i in range(5,96)],axis=0)).reset_index()#生成管道编号
ppcode=ppcode['编号']
pplabel=(pd.concat([pp.loc[range(0,68),['编号']] for i in range(5,96)],axis=0)-1).reset_index()#生成管道标签
pplabel=pplabel['编号']

ppc=pd.concat([ppcode,pplabel,ppdia,ppcoe],axis=1)
ppc.columns=['Code','label','InnerDiameter','DN']

file_path='E:\\Anaconda3\\PyComDH\\out\\'
#获取堵塞数据
num=[ ]
col=pd.DataFrame()
for i in tqdm(range(0,6188)):
    if os.path.exists(file_path+str(i)+'\\Result.6.UserM.csv'):
        Puser=pd.read_csv(file_path+str(i)+'\\Result.6.UserM.csv',header=0,usecols=[2]).T.reset_index(drop=True)#用户的进口压力
        Pheat=pd.read_csv(file_path+str(i)+'\\Result.0.Heater.csv',header=0,usecols=[6]).T.reset_index(drop=True)#加热器的出口压力
        Qheat=pd.read_csv(file_path+str(i)+'\\Result.0.Heater.csv',header=0,usecols=[9]).T.reset_index(drop=True)#加热器的出口流量
        Ppump=pd.read_csv(file_path+str(i)+'\\Result.8.PumpOL.csv',header=0,usecols=[2]).T.reset_index(drop=True)#增压泵的进口压力（补水泵定压）
        Ppipe=pd.read_csv(file_path+str(i)+'\\Result.1.Pipe.csv',header=0,usecols=[2]).T.reset_index(drop=True)#供水管道节点压力
        DP=pd.read_csv(file_path+str(i)+'\\Result.6.UserM.csv',header=0,usecols=[10]).T.reset_index(drop=True)#用户的压降
        cat=pd.concat([Puser,Pheat,Qheat,Ppump,Ppipe,DP],axis=1)#横向拼接
        col=pd.concat([col,cat],axis=0)#纵向拼接
        num.append(i)#记下有结果的编号
Pipe_valid=ppc.iloc[num,:]#留下有结果的样本
Pipe_valid=Pipe_valid.reset_index(drop=False)
col=col.reset_index(drop=True)
col.columns=(['Puser'+str(i) for i in range(1,11)]+
             ['Pheat'+str(i) for i in range(1,3)]+
             ['Qheat'+str(i) for i in range(1,3)]+
             ['Ppump']+
             ['Ppipe'+str(i) for i in range(1,71)]+
            ['DP'+str(i) for i in range(1,11)])

#获取正常数据
Puser=pd.read_csv('E:\\Anaconda3\\PyComDH\\normal\\Result.6.UserM.csv',header=0,usecols=[2]).T.reset_index(drop=True)#用户的进口压力
Pheat=pd.read_csv('E:\\Anaconda3\\PyComDH\\normal\\Result.0.Heater.csv',header=0,usecols=[6]).T.reset_index(drop=True)#加热器的出口压力
Qheat=pd.read_csv('E:\\Anaconda3\\PyComDH\\normal\\Result.0.Heater.csv',header=0,usecols=[9]).T.reset_index(drop=True)#加热器的出口流量
Ppump=pd.read_csv('E:\\Anaconda3\\PyComDH\\normal\\Result.8.PumpOL.csv',header=0,usecols=[2]).T.reset_index(drop=True)#增压泵的进口压力（补水泵定压）
Ppipe=pd.read_csv('E:\\Anaconda3\\PyComDH\\normal\\Result.1.Pipe.csv',header=0,usecols=[2]).T.reset_index(drop=True)
DP=pd.read_csv('E:\\Anaconda3\\PyComDH\\normal\\Result.6.UserM.csv',header=0,usecols=[10]).T.reset_index(drop=True)#用户的压降
col2=pd.concat([Puser,Pheat,Qheat,Ppump,Ppipe,DP],axis=1)
col2=col2.reset_index(drop=True)
col2.columns=(['Puser'+str(i) for i in range(1,11)]+
             ['Pheat'+str(i) for i in range(1,3)]+
             ['Qheat'+str(i) for i in range(1,3)]+
             ['Ppump']+
             ['Ppipe'+str(i) for i in range(1,71)]+
            ['DP'+str(i) for i in range(1,11)])

# 做残差，取变化百分比
samp_p=(col-np.array(col2))/np.array(col2)
# samp_p=col-np.array(col2)
samp=pd.concat([samp_p,Pipe_valid],axis=1)

a=np.array([ ])
for i in range(85,95):
    ind=np.where(col.iloc[:,i]<=0)[0].tolist()#找到用户压降为负的
    a=np.hstack([a,ind]) 
b=np.unique(a).astype(int)

samp=samp.drop(index=b).reset_index(drop=True)
# samp=samp.iloc[:,[i for i in range(0,21)]+[i for i in range(22,27)]+[i for i in range(31,37)]+[i for i in range(47,52)]]
samp.to_csv('E:\\Anaconda3\\PyComDH\\sample_percent.csv',index=False)

100%|██████████| 6188/6188 [00:50<00:00, 121.43it/s]


In [123]:
samp=samp.iloc[:,[i for i in range(0,21)]+[i for i in range(22,27)]+[i for i in range(31,37)]+[i for i in range(47,52)]]

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,index,Code,label,InnerDiameter,DN
0,-0.019792,-0.023421,0.033059,-0.024751,-0.034596,-0.024714,-0.028467,-0.033032,-0.034503,-0.027265,...,-0.027945,-0.025534,-0.024008,-0.027471,-0.025683,1,2,1,0.0225,0.05
1,0.000932,0.000435,0.004724,0.000379,0.003268,-0.000725,-0.008209,-0.024610,-0.031597,-0.004440,...,0.001374,0.000880,0.000232,-0.004716,-0.003450,2,3,2,0.0225,0.05
2,-0.001392,-0.001019,-0.003961,-0.001008,-0.004312,0.000157,0.007948,0.015922,-0.010120,0.003330,...,-0.001829,-0.001406,-0.000832,0.003490,0.002665,3,4,3,0.0225,0.05
3,-0.006279,-0.004869,-0.018500,-0.004882,-0.019228,0.000808,0.035175,-0.062034,-0.052221,0.013750,...,-0.007337,-0.006203,-0.003995,0.014180,0.011535,4,5,4,0.0225,0.05
4,-0.026319,-0.035822,-0.007440,-0.034896,-0.005464,-0.048183,0.014700,0.003081,-0.001688,0.025699,...,-0.006512,-0.025074,-0.037087,0.016471,0.043007,7,8,7,0.0225,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4573,-0.000150,-0.000154,-0.000144,-0.000155,-0.000154,-0.000158,-0.000164,-0.000159,-0.000157,-0.000163,...,-0.000154,-0.000152,-0.000154,-0.000161,-0.000163,6183,64,63,0.4275,0.95
4574,0.000042,0.000043,0.000040,0.000043,0.000043,0.000044,0.000046,0.000045,0.000044,0.000046,...,0.000043,0.000043,0.000043,0.000045,0.000046,6184,65,64,0.4275,0.95
4575,-0.000028,-0.000029,-0.000027,-0.000029,-0.000029,-0.000030,-0.000031,-0.000030,-0.000030,-0.000031,...,-0.000029,-0.000029,-0.000029,-0.000030,-0.000031,6185,66,65,0.4275,0.95
4576,-0.000349,-0.000357,-0.000334,-0.000359,-0.000357,-0.000366,-0.000381,-0.000370,-0.000365,-0.000378,...,-0.000357,-0.000353,-0.000357,-0.000373,-0.000378,6186,67,66,0.4275,0.95
